# Written Report 

# ☆ Charter schools stand out with higher average scores and a greater percentage of students passing in both math and reading. 
# ☆ Schools with a student population of 2000 or less and those with a budget of $615 or lower per student demonstrate higher average scores and passing rates. 
# ☆ Average math and reading scores remain relatively consistent across different grade levels when grouped by school. However, math passing rates consistently lag behind reading passing rates, particularly in lower-performing schools, large schools, and those with higher spending per student. 
# ☆ The top-performing schools are all charters, while the bottom-performing schools are all district schools. Surprisingly, higher spending per student does not necessarily translate to better test results. 
# ☆ Smaller and medium-sized schools outperform larger schools in passing math performances significantly. Charter schools outperform public district schools across all metrics, but further analysis is required to determine whether this effect is due to school practices or smaller student populations per school. 
# ☆ Overall, these insights provide valuable information for educators and the city's school district to make informed decisions in improving educational outcomes for students.

In [73]:
# Import dependencies and setup 
import pandas as pd
from pathlib import Path

# Files to load
school_data_to_load = "Resources\schools_complete.csv"
student_data_to_load = "Resources\students_complete.csv"

# Read school and student data file and store into Pandas DataFrames 
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single data set 
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [74]:
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


# District Summary 

In [75]:
# Calculate the total schools and students 
school_count = len(school_data_complete["school_name"].unique())
student_count = school_data_complete["Student ID"].count()

# Calculate the total budget
total_budget = school_data['budget'].sum()

# Use the sum and mean methods to determine budget and average score values
average_math_score = school_data_complete["math_score"].mean()
average_reading_score = school_data_complete["reading_score"].mean()
overall_passing_rate = (average_math_score+average_reading_score)/2

# Create a filtered data frame to find rows with scores over 70 for each subject
    #Then, divide that amount by the full amount of students passing each subject 
passing_math_count = school_data_complete[(school_data_complete["math_score"] > 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100 
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] > 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100

# Create a data frame and clean up data
district_summary = pd.DataFrame({"Total Schools": [school_count],
                                "Total Students": [student_count],
                                "Total Budget": [total_budget],
                                "Average Math Score": [average_math_score],
                                "Average Reading Score": [average_reading_score],
                                "% Passing Math": [passing_math_percentage],
                                "% Passing Reading": [passing_reading_percentage],
                                "% Overall Passing Rate": [overall_passing_rate]
                                })

# Format the columns to display 
district_summary = district_summary[["Total Schools", "Total Students", "Total Budget", 
                                     "Average Math Score", "Average Reading Score",
                                     "% Passing Math", "% Passing Reading", "% Overall Passing Rate"
                                    ]]

district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,72.392137,82.971662,80.431606


# School Summary 

In [76]:
# Determine the school type 
school_types = school_data.set_index(["school_name"])["type"]

# Calculate the total student count 
per_school_counts = school_data_complete["school_name"].value_counts()

# Calculate the total school budget and per capita spending per school
per_school_budget = school_data_complete.groupby(["school_name"])["budget"].mean()
per_school_capita = per_school_budget / per_school_counts

# Calculate the average test scores 
per_school_math = school_data_complete.groupby(["school_name"])["math_score"].mean()
per_school_reading = school_data_complete.groupby(["school_name"])["reading_score"].mean()

# Calculate the passing scores by creating a filtered DataFrame 
school_passing_math = school_data_complete[(school_data_complete["math_score"] > 70)]
school_passing_reading = school_data_complete[(school_data_complete["reading_score"] > 70)]

per_school_passing_math = school_passing_math.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100 
per_school_passing_reading = school_passing_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100 
overall_passing_rate = (per_school_passing_math + per_school_passing_reading) / 2 

# Convert to DataFrame 
per_school_summary = pd.DataFrame({"School Type": school_types,
                                  "Total Students": per_school_counts,
                                  "Total School Budget": per_school_budget,
                                  "Per Student Budget": per_school_capita,
                                  "Average Math Score": per_school_math,
                                  "Average Reading Score": per_school_reading,
                                  "% Passing Math": per_school_passing_math,
                                  "% Passing Reading": per_school_passing_reading,
                                  "% Overall Passing Rate": overall_passing_rate
                                  })

# Format the columns to display 
per_school_summary = per_school_summary[["School Type", "Total Students", "Total School Budget", "Per Student Budget",
                                         "Average Math Score", "Average Reading Score", "% Passing Math", 
                                         "% Passing Reading", "% Overall Passing Rate"]]

per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame 
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,64.630225,79.300643,71.965434
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,89.558665,93.864370,91.711518
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,63.750424,78.433367,71.091896
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,65.753925,77.510040,71.631982
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,89.713896,93.392371,91.553134
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,64.746494,78.187702,71.467098
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,90.632319,92.740047,91.686183
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,63.318478,78.813850,71.066164
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,63.852132,78.281874,71.067003


# Top 5 Performing Schools

In [77]:
# Sort and show top 5 schools 
top_schools = per_school_summary.sort_values(["% Overall Passing Rate"], ascending=False)
top_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,90.932983,93.254490,92.093736
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,91.683992,92.203742,91.943867
Wright High School,Charter,1800,"$1,049,400.00",$583.00,83.682222,83.955000,90.277778,93.444444,91.861111
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,89.558665,93.864370,91.711518
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,90.632319,92.740047,91.686183


# Bottom 5 Performing Schools

In [78]:
# Sort and show bottom 5 schools 
bottom_schools = per_school_summary.sort_values(["% Overall Passing Rate"], ascending=True)
bottom_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,64.066017,77.744436,70.905226
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,63.318478,78.813850,71.066164
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,63.852132,78.281874,71.067003
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,63.750424,78.433367,71.091896
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,64.746494,78.187702,71.467098


# Math Scores by Grade

In [79]:
# Select data for each grade level
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Calculate the average math scores for each grade
ninth_graders_scores = ninth_graders.groupby(["school_name"])["math_score"].mean()
tenth_graders_scores = tenth_graders.groupby(["school_name"])["math_score"].mean()
eleventh_graders_scores = eleventh_graders.groupby(["school_name"])["math_score"].mean()
twelfth_graders_scores = twelfth_graders.groupby(["school_name"])["math_score"].mean()

# Create our DataFrame 
scores_by_grade = pd.DataFrame({"9th": ninth_graders_scores, "10th": tenth_graders_scores, 
                                "11th": eleventh_graders_scores, "12th": twelfth_graders_scores})

# Clean up our DataFrame and make sure the columns show up in the correct order  
scores_by_grade = scores_by_grade[["9th", "10th", "11th", "12th"]]

# Display the DataFrame 
scores_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


# Reading Scores by Grade

In [80]:
# Select data for each grade level
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Calculate the average math scores for each grade
ninth_graders_scores = ninth_graders.groupby(["school_name"])["reading_score"].mean()
tenth_graders_scores = tenth_graders.groupby(["school_name"])["reading_score"].mean()
eleventh_graders_scores = eleventh_graders.groupby(["school_name"])["reading_score"].mean()
twelfth_graders_scores = twelfth_graders.groupby(["school_name"])["reading_score"].mean()

# Create our DataFrame 
scores_by_grade = pd.DataFrame({"9th": ninth_graders_scores, "10th": tenth_graders_scores, 
                                "11th": eleventh_graders_scores, "12th": twelfth_graders_scores})

# Clean up our DataFrame and make sure the columns show up in the correct order  
scores_by_grade = scores_by_grade[["9th", "10th", "11th", "12th"]]

# Display the DataFrame 
scores_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


# Scores by Spending 

In [81]:
# Establish the ranges for our bins 
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<585", "585-615", "615-645", "645-675"]

# Find the average values for each range 
per_school_summary["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=group_names)

spending_math_scores = per_school_summary.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = per_school_summary.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = per_school_summary.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = per_school_summary.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_rate = (spending_math_scores + spending_reading_scores) / 2 

# Assemble the DataFrame 
scores_by_spending_summary = pd.DataFrame({"Average Math Score": spending_math_scores, 
                                "Average Reading Score": spending_reading_scores,
                                "% Passing Math": spending_passing_math, 
                                "% Passing Reading": spending_passing_reading, 
                                "% Overall Passing Rate": overall_passing_rate})

# Make sure the DataFrame is being displayed in the order we want 
scores_by_spending_summary = scores_by_spending_summary[["Average Math Score", "Average Reading Score", 
                                    "% Passing Math", "% Passing Reading", 
                                    "% Overall Passing Rate"]]

# Display the DataFrame 
scores_by_spending_summary



,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<585,83.455399,83.933814,90.350436,93.325838,83.694607
585-615,83.599686,83.885211,90.788049,92.410786,83.742449
615-645,79.079225,81.891436,73.021426,83.214343,80.485330
645-675,76.997210,81.027843,63.972368,78.427809,79.012526


# Scores by School Size 

In [82]:
# Establish the ranghes for our bins 
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Find the average values for each range 
per_school_summary["School Size"] = pd.cut(per_school_summary["Total Students"], size_bins, labels=group_names)

# Calculate the scores based on bins 
size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
overall_passing_rate = (size_passing_math + size_passing_reading) / 2 

# Assemble into DataFrame 
scores_by_size_summary  = pd.DataFrame({"Average Math Score": size_math_scores,
                             "Average Reading Score": size_reading_scores,
                             "% Passing Math": size_passing_math,
                             "% Passing Reading": size_passing_reading,
                             "% Overall Passing Rate": overall_passing_rate})

# Make sure the DataFrame is being displayed in the order we want 
scores_by_size_summary  = scores_by_size_summary [["Average Math Score", "Average Reading Score", 
                             "% Passing Math", "% Passing Reading", 
                             "% Overall Passing Rate"]]

# Display the DataFrame 
scores_by_size_summary 


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,91.158155,92.471895,91.815025
Medium (1000-2000),83.374684,83.864438,89.931303,93.244843,91.588073
Large (2000-5000),77.746417,81.344493,67.631335,80.190800,73.911067


# Scores by School Type 

In [83]:
# Calculate the scores based on bins 
type_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
type_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
type_passing_math = per_school_summary.groupby(["School Size"])["% Passing Math"].mean()
type_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
overall_passing_rate = (type_passing_math + type_passing_reading) / 2 

# Assemble into DataFrame 
scores_by_type_summary = pd.DataFrame({"Average Math Score": type_math_scores,
                                       "Average Reading Score": type_reading_scores,
                                       "% Passing Math": type_passing_math,
                                       "% Passing Reading": type_passing_reading,
                                       "% Overall Passing Rate": overall_passing_rate})

# Make sure the DataFrame is being displayed in the order we want 
scores_by_type_summary = scores_by_type_summary[["Average Math Score", "Average Reading Score", 
                                                 "% Passing Math", "% Passing Reading", 
                                                 "% Overall Passing Rate"]]

# Display the DataFrame 
scores_by_type_summary 


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,91.158155,92.471895,91.815025
Medium (1000-2000),83.374684,83.864438,89.931303,93.244843,91.588073
Large (2000-5000),77.746417,81.344493,67.631335,80.190800,73.911067
